<a name="top"></a><img src="images/chisel_1024.png" alt="Chisel logo" style="width:480px;" />

# Module 2.5: Putting it all Together: An FIR Filter
**Prev: [Sequential Logic](2.4_sequential_logic.ipynb)**<br>
**Next: [Generators: Parameters](3.1_parameters.ipynb)**

## Motivation
Now that you've learned the basics of Chisel, let's use that knowledge to build a FIR (finite impulse response) filter module! FIR filters are very common in digital signal processing applications. Also, the FIR filter will reappear frequently in module 3, so it's important that you don't filter out this module by skipping ahead! If you are unfamiliar with FIR filters, head over to the article on [trusty Wikipedia](https://en.wikipedia.org/wiki/Finite_impulse_response) to learn more.

## Setup

In [ ]:
val path = System.getProperty("user.dir") + "/source/load-ivy.sc"
interp.load.module(ammonite.ops.Path(java.nio.file.FileSystems.getDefault().getPath(path)))

In [ ]:
import chisel3._
import chisel3.util._
import chisel3.iotesters.{ChiselFlatSpec, Driver, PeekPokeTester}

---
# FIR Filter

The FIR filter you will design performs the following operation.

<img src="images/fir.jpg" width="720">

Basically, this does a elementwise multiplication of the element of the filter coefficients with the elements of the input signal and outputs the sum (also called a _convolution_).

Or, a signals definition:

$y[n] = b_0 x[n] + b_1 x[n-1] + b_2 x[n-2] + ...$
 - $y[n]$ is the output signal at time $n$
 - $x[n]$ is the input signal
 - $b_i$ are the filter coefficients or impulse response
 - $n-1$, $n-2$, ... are time $n$ delayed by 1, 2, ... cycles
 
## 8-bit Specification

Build a 4-element FIR filter where the four filter coefficients are parameters. A module skeleton and basic tests are provided for you.
Note that both the input and output are 8-bit unsigned integers. You will need to save necessary state (like delayed signal values) using constructs like shift registers. Use the provided testers to check your implementation.
Registers with constant inputs can be created using a `ShiftRegister` of shift value 1, or by using the `RegNext` construct.

Note: for the tests to pass, your registers must be initialized to `0.U`.

In [ ]:
class My4ElementFir(b0: Int, b1: Int, b2: Int, b3: Int) extends Module {
  val io = IO(new Bundle {
    val in = Input(UInt(8.W))
    val out = Output(UInt(8.W))
  })

  ???
}

In [ ]:
// Simple sanity check: a element with all zero coefficients should always produce zero
Driver(() => new My4ElementFir(0, 0, 0, 0)) {
  c => new PeekPokeTester(c) {
    poke(c.io.in, 0)
    expect(c.io.out, 0)
    step(1)
    poke(c.io.in, 4)
    expect(c.io.out, 0)
    step(1)
    poke(c.io.in, 5)
    expect(c.io.out, 0)
    step(1)
    poke(c.io.in, 2)
    expect(c.io.out, 0)
  }
}

In [ ]:
// Simple 4-point moving average
Driver(() => new My4ElementFir(1, 1, 1, 1)) {
  c => new PeekPokeTester(c) {
    poke(c.io.in, 1)
    expect(c.io.out, 1)  // 1, 0, 0, 0
    step(1)
    poke(c.io.in, 4)
    expect(c.io.out, 5)  // 4, 1, 0, 0
    step(1)
    poke(c.io.in, 3)
    expect(c.io.out, 8)  // 3, 4, 1, 0
    step(1)
    poke(c.io.in, 2)
    expect(c.io.out, 10)  // 2, 3, 4, 1
    step(1)
    poke(c.io.in, 7)
    expect(c.io.out, 16)  // 7, 2, 3, 4
    step(1)
    poke(c.io.in, 0)
    expect(c.io.out, 12)  // 0, 7, 2, 3
  }
}

In [ ]:
// Nonsymmetric filter
Driver(() => new My4ElementFir(1, 2, 3, 4)) {
  c => new PeekPokeTester(c) {
    poke(c.io.in, 1)
    expect(c.io.out, 1)  // 1*1, 0*2, 0*3, 0*4
    step(1)
    poke(c.io.in, 4)
    expect(c.io.out, 6)  // 4*1, 1*2, 0*3, 0*4
    step(1)
    poke(c.io.in, 3)
    expect(c.io.out, 14)  // 3*1, 4*2, 1*3, 0*4
    step(1)
    poke(c.io.in, 2)
    expect(c.io.out, 24)  // 2*1, 3*2, 4*3, 1*4
    step(1)
    poke(c.io.in, 7)
    expect(c.io.out, 36)  // 7*1, 2*2, 3*3, 4*4
    step(1)
    poke(c.io.in, 0)
    expect(c.io.out, 32)  // 0*1, 7*2, 2*3, 3*4
  }
}

<div id="container"><section id="accordion"><div>
<input type="checkbox" id="check-1" />
<label for="check-1"><strong>Solution</strong></label>
<article>
<pre style="background-color:#f7f7f7">
  val x_n1 = RegNext(io.in, 0.U)
  val x_n2 = RegNext(x_n1, 0.U)
  val x_n3 = RegNext(x_n2, 0.U)
  io.out := io.in \* b0.U(8.W) + 
    x_n1 \* b1.U(8.W) +
    x_n2 \* b2.U(8.W) + 
    x_n3 \* b3.U(8.W)
</pre></article></div></section></div>

---
# IPXact Example

Your Chisel generator produces Verilog, but just handing someone else a piece of Verilog is insufficient to verify and integrate the design. What other information is needed to capture a design’s intent? IPXact solves this by providing an XML description of a design and its metadata, including the interfaces, parameters, address mapping, etc. So for this portion of the lab, we'll standardize the FIR interfaces and write out IPXact to ease implementation and verification. You'll need the results of this generator in later labs.

## Setup
We have compiled the necessary depedencies, which are not all published on Maven, into a jar file. Contact the contributors if you want a copy. If you are not at the bootcamp, you can safely skip this section.

In [ ]:
// depdencies for ipxact
val path = System.getProperty("user.dir") + "/../rocket-dsp-utils-assembly-1.0.jar"
interp.load.cp(ammonite.ops.Path(java.nio.file.FileSystems.getDefault().getPath(path)))

---
# FIR Filter Generator

For this module, we'll be using a slightly modified example from [Module 3.2: Generators: Collection](3.2_collections.ipynb).
If you haven't started Module 3.2, don't worry.
You'll learn about the details of how `MyManyDynamicElementVecFir` works, but the basic idea is that it is a FIR filter generator.

The generator has one parameter: length.
That parameter dictates how many taps the filter has, and the taps are inputs to the hardware `Module`.

The generator has 3 inputs:
* in, the input to the filter
* valid, a boolean that says when the input is valid
* consts, a vector for all the taps

and 1 output:
* out, the filtered input

<img src="images/fir.jpg" style="width:450px;"/>

In [ ]:
class MyManyDynamicElementVecFir(length: Int) extends Module {
  val io = IO(new Bundle {
    val in = Input(UInt(8.W))
    val valid = Input(Bool())
    val out = Output(UInt(8.W))
    val consts = Input(Vec(length, UInt(8.W)))
  })
  
  // Such concision! You'll learn what all this means later.
  val taps = Seq(io.in) ++ Seq.fill(io.consts.length - 1)(RegInit(0.U(8.W)))
  taps.zip(taps.tail).foreach { case (a, b) => when (io.valid) { b := a } }

  io.out := taps.zip(io.consts).map { case (a, b) => a * b }.reduce(_ + _)
}

---
# DspBlock

Integrating DSP components into a larger system can be challenging and error prone.
The [rocket-dsp-utils](https://github.com/ucb-art/rocket-dsp-utils) repository consists of useful generators that should help with such tasks.

One of the core abstractions is the notion of a DSPBlock.
A DSPBlock has:
* AXI-4 Stream input and output
* AXI-4 memory-mapped status and control


<img src="images/fir_filter.png" style="width:800px;"/>

The following code wraps the FIR filter in AXI4 interfaces.

In [ ]:
import dspblocks._
import freechips.rocketchip.amba.axi4._
import freechips.rocketchip.amba.axi4stream._
import freechips.rocketchip.config._
import freechips.rocketchip.diplomacy._

case object NumFilters extends CSRField {
    val name = "firQueueDepth"
}

case class FIRTap(filterIdx: Int, tapIdx: Int) extends CSRField {
    val name = s"firTap${filterIdx}_$tapIdx"
}

class FIRBlock(nFilters: Int, nTaps: Int)(implicit p: Parameters) extends
AXI4DspBlock with AXI4HasCSR with HasIPXactParameters {
    outer =>
    addStatus(NumFilters)
    
    for (i <- 0 until nFilters) {
        for (j <- 0 until nTaps) {
            addControl(FIRTap(i, j))
        }
    }
    makeCSRs()
    
    val streamNode = AXI4StreamIdentityNode()
    
    override def ipxactParameters = Map(
        "nFilters"   -> nFilters.toString,
        "nTaps"      -> nTaps.toString
    )

    def beatBytes: Int = 8
    def csrAddress = freechips.rocketchip.diplomacy.AddressSet(0x0, 0xffffL)
    def csrBase: Int = 0
    def csrSize: Int = 4096
    
    lazy val module = new LazyModuleImp(this) {
        val (in, _)  = streamNode.in(0)
        val (out, _) = streamNode.out(0)
        val mem = outer.mem.map { m => m.in(0) }
        
        require(in.params.n >= nFilters,
                s"""AXI-4 Stream port must be big enough for all 
                   |the filters (need $nFilters,, only have ${in.params.n})""".stripMargin)

        status(NumFilters) := nFilters.U
        
        
        val outs = (0 until nFilters).map(i => {
            val fir = Module(new MyManyDynamicElementVecFir(nTaps))
            
            fir.io.in := in.bits.data((i+1)*8, i*8)
            fir.io.valid := in.valid && out.ready
            
            for (j <- 0 until nTaps) {
                fir.io.consts(j) := control(FIRTap(i, j))
            }
            
            fir.io.out
        })
        
        
        val output = if (outs.length == 1) {
            outs(0)
        } else {
            outs.reduce((x: UInt, y: UInt) => Cat(y, x))
        }
            
        out.bits.data := output
        in.ready  := out.ready
        out.valid := in.valid
    }
}

---
# Invoking the Generator

The following code invokes the generator.
It will produce a file called `BlindModule.fir` which contains the firrtl generated by our code, as well as a file called `BlindModule.v` which is the compiled verilog.
`BlindModule` is a wrapper for our `DspBlock`- it is necessary because of how [diplomacy](https://carrv.github.io/papers/cook-diplomacy-carrv2017.pdf) in rocket works.
You'll notice that `BlindModule` instantiates an `FIRBlock`.

The line
```scala
AXI4StreamBundleParameters(n = 8)
```
sets the AXI-4 streams' data widths to 8 bytes.

In [ ]:
object FIRGenerator extends ipxact.IPXactGeneratorApp with App {
  override val verilogFilename: String = "BlindModule.v"
  override val ipxactDir: String = "./"
  implicit val p: Parameters = Parameters.root((new freechips.rocketchip.coreplex.BaseCoreplexConfig).toInstance)

    val blindNodes = DspBlockBlindNodes.apply(
      AXI4StreamBundleParameters(n = 8),
      () => AXI4MasterNode(Seq(AXI4MasterPortParameters(Seq(AXI4MasterParameters("fir"))))))
    
    val dut = () => {
        val lazyMod = LazyModule(DspBlock.blindWrapper(() => new FIRBlock(4, 8), blindNodes))
        val m = lazyMod.module
        IPXactComponents._ipxactComponents += DspIPXact.makeDspBlockComponent(lazyMod.internal)
        m
    }

    
    chisel3.Driver.dumpFirrtl(chisel3.Driver.elaborate(dut), None)
    chisel3.Driver.compileFirrtlToVerilog("BlindModule", new java.io.File(System.getProperty("user.dir")))
    generateIPXact(IPXactComponents.ipxactComponents())
}
FIRGenerator.main(Array[String]())

---
# IPXact
Look in the file browser for a file called `craft_BlindModule_edu.berkeley.cs_1.0.xml`.
This is the ipxact file.
It contains information about:
* Port mappings
* Interfaces
* Memory maps
* Generator parameters

You'll notice that the parameters from the scala code
```scala
override def ipxactParameters = Map(
    "nFilters"   -> nFilters.toString,
    "nTaps"      -> nTaps.toString
)
```
appears in the IPXact output.
This gives verification generators the information they need to generate appropriate test vectors for the given instance.

---
# You're done!

[Return to the top.](#top)